
**Part 1: Setup & Model Loading**

In [61]:
#  Install HuggingFace Transformers
!pip install transformers

#  Import libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

#  Load GPT-2 model and tokenizer
model_name = "gpt2-medium"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

print(" GPT-2 model loaded and ready.")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

 GPT-2 model loaded and ready.


In [62]:
!pip install dash



** Part 2: Prompt + Script Generator Function**

In [63]:
def generate_script(prompt, max_length=300, temperature=0.8):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            no_repeat_ngram_size=2,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)


**Part 3: Test the Script Generator**

In [64]:
def generate_script(prompt, max_length=500, temperature=0.8):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            no_repeat_ngram_size=2,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)
# Generate script
script_output = generate_script(genre_prompt)

# Print the result
print("Generated Script:\n")
print(script_output)


Generated Script:

INT. ABANDONED HOSPITAL - NIGHT
A group of teenagers enters the dark hallway. Suddenly, they hear a whisper echoing through the walls... "FUCK YOU!!"
THE BOOGIE SUSPECT - THAT NIGHT He is lying on the floor with his back to the door. There is nothing else he can do. He does not even feel the cold. The room is completely dark.
The girl next to him hears the whispered whisper and stops in her tracks. She turns around to see a small figure standing on a bed. This tiny figure looks exactly the same as the one he saw earlier. "It's your mother, my baby." She points at the tiny person.  "We are the only ones left here."
"That's not true. I have a brother that has left me." The little boy looks at her strangely. "...And I got a friend with me who is probably hiding somewhere." He looks around, looking for something. Something that will help him get through his darkest hour. As she looks down at him, she can't help but shake with fear. What is happening to her? 
After severa

**Multiple Genre Options + Script Formatting + Customizable :**

In [65]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import random

# Sample layout for the Dash app
app = dash.Dash(__name__)

# Function to generate the script with word count matching
def generate_script(genre, tone, characters, length, prompt, continue_scene=""):
    structured_prompt = (
        f"Genre: {genre}\n"
        f"Tone: {tone}\n"
        f"Characters: {characters}\n\n"
    )

    if continue_scene:
        structured_prompt += f"Previous Scene:\n{continue_scene}\n\nContinue the story:\n"

    # Placeholder story structure
    base_script = (
        f"🎬 Generated {genre} Script:\n\n"
        f"Genre: {genre}\n"
        f"Tone: {tone}\n"
        f"Characters: {characters}\n\n"
        f"Length: {length} words\n\n"
        f"Story Prompt: {prompt}\n\n"
        f"------------------------------------\n"
        f"Scene begins...\n\n"
        f"{characters} is a {genre} character caught in a {tone} situation.\n\n"
        f"With the stakes high, {characters} must {prompt} to survive the danger...\n\n"
        f"(Continue the scene...)\n\n"
        f"------------------------------------\n"
        f"(This is a placeholder for your generated content. In a real setup, this would be the AI-generated text.)"
    )

    # Helper function to count words
    def count_words(text):
        return len(text.split())

    # Generate content
    generated_content = base_script
    words_count = count_words(generated_content)

    # Simulate generating the story (you can replace this with AI generation logic later)
    while words_count < length:
        # Adding more content based on the characters and plot
        generated_content += f"\n{characters} faces a new challenge. The danger intensifies as {characters} must make crucial decisions.\n"
        words_count = count_words(generated_content)

        # Prevent infinite loops
        if words_count >= length:
            break

    # If generated content exceeds length, trim it
    if words_count > length:
        words = generated_content.split()
        generated_content = " ".join(words[:length])  # Trim excess words

    return generated_content


# Layout of the app
app.layout = html.Div([
    html.H1("Movie Script Generator"),

    # Genre dropdown
    dcc.Dropdown(
        id='genre-dropdown',
        options=[
            {'label': 'Horror', 'value': 'Horror'},
            {'label': 'Romance', 'value': 'Romance'},
            {'label': 'Action', 'value': 'Action'},
            {'label': 'Sci-Fi', 'value': 'Sci-Fi'},
            {'label': 'Comedy', 'value': 'Comedy'}
        ],
        value='Horror',
        style={'width': '50%', 'margin': 'auto'}
    ),

    # Tone dropdown
    dcc.Dropdown(
        id='tone-dropdown',
        options=[
            {'label': 'Suspenseful', 'value': 'Suspenseful'},
            {'label': 'Funny', 'value': 'Funny'},
            {'label': 'Romantic', 'value': 'Romantic'},
            {'label': 'Dark', 'value': 'Dark'},
            {'label': 'Light-hearted', 'value': 'Light-hearted'},
            {'label': 'Epic', 'value': 'Epic'}
        ],
        value='Suspenseful',
        style={'width': '50%', 'margin': 'auto'}
    ),

    # Characters input
    dcc.Input(id='characters-input', type='text', placeholder="e.g., John (hero), Mira (detective)", style={'width': '50%', 'margin': 'auto'}),

    # Length slider
    dcc.Slider(
        id='length-slider',
        min=100,
        max=1000,
        step=50,
        value=300,
        marks={i: str(i) for i in range(100, 1001, 50)},
        className='slider',
    ),

    # Prompt input
    dcc.Textarea(
        id='prompt-input',
        placeholder='Type your movie scene prompt here...',
        style={'width': '50%', 'height': '150px', 'margin': 'auto'}
    ),

    # Continue scene input (optional)
    dcc.Textarea(
        id='continue-input',
        placeholder='Paste the last scene here if you want to continue it (optional)...',
        style={'width': '50%', 'height': '100px', 'margin': 'auto'}
    ),

    # Button to generate the script
    html.Button('Generate Script 🎬', id='generate-button', n_clicks=0, style={'display': 'block', 'margin': 'auto'}),

    # Output area for generated script
    html.Div([
        dcc.Loading(
            id="loading",
            type="circle",
            children=[
                html.Div(id='output-script', style={'white-space': 'pre-wrap', 'padding': '10px', 'background': '#f0f0f0', 'margin-top': '20px'})
            ])]),
])


# Callback for generating script
@app.callback(
    Output('output-script', 'children'),
    [Input('generate-button', 'n_clicks')],
    [
        dash.dependencies.State('genre-dropdown', 'value'),
        dash.dependencies.State('tone-dropdown', 'value'),
        dash.dependencies.State('characters-input', 'value'),
        dash.dependencies.State('length-slider', 'value'),
        dash.dependencies.State('prompt-input', 'value'),
        dash.dependencies.State('continue-input', 'value')
    ]
)
def generate_script_callback(n_clicks, genre, tone, characters, length, prompt, continue_scene):
    if n_clicks > 0 and prompt:
        script = generate_script(genre, tone, characters, length, prompt, continue_scene)
        return script
    return "Enter a prompt and click 'Generate Script 🎬'"

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>